In [2]:
import os
from PIL import Image
import numpy as np
import cv2

In [3]:
def vectorizar(carpeta, label):
    archivos = os.listdir(carpeta)

    # Lista para almacenar los vectores de las imágenes
    vectores_imagenes = []

    # Iterar sobre los archivos
    for archivo in archivos:
        # Comprobar si es un archivo de imagen
        if archivo.endswith(".jpg") or archivo.endswith(".png"):
            # Ruta completa de la imagen
            ruta_imagen = os.path.join(carpeta, archivo)
            
            # Cargar la imagen
            imagen = Image.open(ruta_imagen)

            # Redimensiona la imagen a 8x8 píxeles
            imagen = imagen.resize((50, 125))

            # Convierte la imagen a escala de grises
            imagen = imagen.convert("L")

            # Convierte la imagen a un array de NumPy
            array_imagen = np.array(imagen)

            # Aplana el array de imagen a un vector de 1x64
            vector = array_imagen.flatten()
            
            # Crea una lista con la etiqueta y el nombre del archivo
            etiqueta_archivo = [label, archivo]

            # Agrega la lista al final del vector
            vector_con_etiqueta_archivo = np.append(vector, etiqueta_archivo)

            # Agrega el vector a la lista de vectores de imágenes
            vectores_imagenes.append(vector_con_etiqueta_archivo)

    # Convierte la lista de vectores en una matriz de NumPy
    return np.array(vectores_imagenes)


In [4]:
data_yes = vectorizar('carpeta_MitadesYes/', 1)
data_no = vectorizar('carpeta_MitadesNo/', 0)

In [5]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

class KNNClassifier:
    def __init__(self, k):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        y_pred = []
        for i in range(len(X_test)):
            distances = []
            for j in range(len(self.X_train)):
                dist = euclidean_distance(X_test[i], self.X_train[j])
                distances.append((dist, self.y_train[j]))
            distances.sort(key=lambda x: x[0])  # Ordenar distancias de menor a mayor
            neighbors = distances[:self.k]  # Obtener los k vecinos más cercanos
            classes = [neighbor[1] for neighbor in neighbors]  # Obtener las clases de los vecinos
            y_pred.append(max(set(classes), key=classes.count))  # Clasificación por voto mayoritario
        return y_pred

In [6]:
def exclude(matriz, indice):
    filas_seleccionadas = []
    for i, fila in enumerate(matriz):
        if i != indice:
            filas_seleccionadas.append(fila)
    return np.array(filas_seleccionadas)

In [7]:
def mergeData(data1, data2):
    return np.concatenate((data1, data2), axis=0)

In [8]:
def LOO_Knn(data_yes, data_no):
    allData = mergeData(data_yes, data_no)
    
    predicted_labels = []
    true_labels = []

In [9]:
allData = mergeData(data_yes, data_no)
print(allData.shape)


(207, 6252)


In [10]:
def LOO_Knn(allData):
    labelsPredictedKNN = []
    labelReal = allData[:, 6250]
    for i in range(allData.shape[0]):
        dataTrain = exclude(allData, i)
        ImgsTrain = np.array(dataTrain[:, :-2], dtype= int)
        LabelsTrain = np.array(dataTrain[:, 6250], dtype= int)
        NameImageTrain = np.array(dataTrain[:, 6251], dtype= str)  


        dataTest = allData[i, :]
        dataTest = dataTest.reshape(1, 6252)
        ImgsTest = np.array(dataTest[:, :-2], dtype=int)
        LabelsTest = np.array(dataTest[:, 6250], dtype=int)
        NameImageTest = np.array(dataTest[:, 6251], dtype=str)

        knn = KNNClassifier(1)
        knn.fit(ImgsTrain, LabelsTrain)
        prediction = knn.predict(ImgsTest)
        
        labelsPredictedKNN.append(prediction[0])
        print(prediction[0], LabelsTest[0])
    labelsPredictedKNN = np.array(labelsPredictedKNN)  
    return(labelsPredictedKNN)

In [11]:
class KMeans:
    def __init__(self, k, max_iterations):
        self.k = k
        self.max_iterations = max_iterations
        self.centroids = None
        self.labels = None

    def initialize_centroids(self, data):
        # Inicialización de centroides de manera aleatoria
        np.random.seed(0)
        indices = np.random.choice(data.shape[0], self.k, replace=False)
        centroids = data[indices]
        return centroids
    
    def initialize_centroids_with_features(self, data, real_labels):
        # Preprocesamiento de las imágenes
        flattened_images = data.reshape(data.shape[0], -1)  # Aplanar las imágenes en vectores de características
        unique_labels = np.unique(real_labels)

        # Cálculo de características promedio por etiqueta
        centroids = []
        for label in unique_labels:
            label_images = flattened_images[real_labels == label]
            label_mean = np.mean(label_images, axis=0)
            centroids.append(label_mean)

        # Selección de los primeros K centroides iniciales
        centroids = np.array(centroids)[:self.k]

        return centroids
    
    def kmeans_plusplus_initialization(self, data, k):
        centroids = []
        centroids.append(data[np.random.choice(data.shape[0])])  # Selecciona el primer centroide aleatoriamente

        for _ in range(1, k):
            distances = np.array([min([np.linalg.norm(point - centroid) for centroid in centroids]) for point in data])
            probabilities = distances / distances.sum()
            next_centroid_index = np.random.choice(data.shape[0], p=probabilities)
            centroids.append(data[next_centroid_index])

        return np.array(centroids)

    def assign_clusters(self, data):
        # Asignación de puntos a clústeres según la distancia euclidiana
        distances = np.sqrt(((data[:, np.newaxis] - self.centroids) ** 2).sum(axis=2))
        labels = np.argmin(distances, axis=1)
        return labels

    def update_centroids(self, data):
        # Actualización de los centroides como la media de los puntos asignados a cada clúster
        centroids = np.zeros((self.k, data.shape[1]))
        for i in range(self.k):
            cluster_data = data[self.labels == i]
            if len(cluster_data) > 0:
                centroids[i] = np.mean(cluster_data, axis=0)
        return centroids

    def fitRANDOM(self, data):
        self.centroids = self.initialize_centroids(data)

        for _ in range(self.max_iterations):
            prev_centroids = self.centroids.copy()

            self.labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)

            # Comprobar convergencia
            if np.all(prev_centroids == self.centroids):
                break
            
    def fitPreprocess(self, data, real_labels):
        self.centroids = self.initialize_centroids_with_features(data, real_labels)

        for _ in range(self.max_iterations):
            prev_centroids = self.centroids.copy()

            self.labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)

            # Comprobar convergencia
            if np.all(prev_centroids == self.centroids):
                break
    
    def fitKplus(self, data):
        self.centroids = self.initialize_centroids_with_features(data, self.k)

        for _ in range(self.max_iterations):
            prev_centroids = self.centroids.copy()

            self.labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)

            # Comprobar convergencia
            if np.array_equal(prev_centroids, self.centroids):
                break

    def predict(self, data):
        labels = self.assign_clusters(data)
        return labels

In [12]:
def LOO_KMeans(allData):
    labelsPre = []
    labelsR = []
    labelsPlus = []
    for i in range(allData.shape[0]):
        dataTrain = exclude(allData, i)
        ImgsTrain = np.array(dataTrain[:, :-2], dtype= int)
        LabelsTrain = np.array(dataTrain[:, 6250], dtype= int)
        NameImageTrain = np.array(dataTrain[:, 6251], dtype= str)  


        dataTest = allData[i, :]
        dataTest = dataTest.reshape(1, 6252)
        ImgsTest = np.array(dataTest[:, :-2], dtype=int)
        LabelsTest = np.array(dataTest[:, 6250], dtype=int)
        NameImageTest = np.array(dataTest[:, 6251], dtype=str)

        iterations, k=2, 2

        kmeansPre = KMeans(k,iterations)
        kmeansR = KMeans(k,iterations)
        kmeansPlus = KMeans(k,iterations)
        # Ajustar el modelo a los datos
        kmeansPre.fitPreprocess(ImgsTrain, LabelsTrain)
        kmeansR.fitRANDOM(ImgsTrain)
        kmeansPlus.fitKplus(ImgsTrain)
        # Obtener las etiquetas asignadas a cada punto
        predicted_labelsPre = kmeansPre.predict(ImgsTest)
        predicted_labelsR = kmeansR.predict(ImgsTest)
        predicted_labelsPlus = kmeansPlus.predict(ImgsTest)
        
        labelsPre.append(predicted_labelsPre[0])
        labelsR.append(predicted_labelsR[0])
        labelsPlus.append(predicted_labelsPlus[0])
    
    accuracyPre = np.mean(labelsPre == np.array(allData[:, 6250], dtype=int))
    accuracyR = np.mean(labelsR == np.array(allData[:, 6250], dtype=int))
    accuracyPlus = np.mean(labelsPlus == np.array(allData[:, 6250], dtype=int))
    
    print(accuracyPre,accuracyR,accuracyPlus)
    

In [13]:
LOO_KMeans(allData)

0.6859903381642513 0.5555555555555556 0.4782608695652174


In [15]:
predictions = LOO_Knn(allData)

0 1
0 1
1 1
1 1
1 1
1 1
1 1
0 1
1 1
1 1
0 1
1 1
0 1
1 1
1 1
0 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 1
0 1
1 1
1 1
1 1
0 1
1 1
0 1
1 1
1 1
1 1
0 1
1 1
1 1
1 1
1 1
1 1
1 1
0 1
1 1
1 1
1 1
1 1
0 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 1
1 1
0 1
0 1
1 1
1 1
0 1
1 1
0 1
1 1
0 1
1 1
1 1
0 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 1
0 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
0 1
1 1
1 1
1 1
1 1
1 1
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
1 0
1 0
0 0
1 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
1 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


In [16]:
print(predictions)
print(np.array(allData[:, 6250], dtype=int))

[0 0 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1
 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0
 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [17]:
accuracy = np.mean(predictions == np.array(allData[:, 6250], dtype=int))
print(accuracy)

0.8164251207729468


In [22]:
NamePredicction =  mergeData(predictions,allData[:, 6251])
NamePredicction = NamePredicction.reshape(2, 207)

['0' '1' '1' '0' '1' '1' '1' '0' '1' '1' '1' '1' '1' '1' '1' '1' '1' '0'
 '1' '0' '0' '1' '0' '1' '1' '1' '1' '1' '0' '1' '1' '1' '0' '0' '1' '0'
 '0' '0' '1' '1' '1' '1' '1' '1' '1' '1' '0' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '0' '0' '0' '0' '0' '1' '0' '1' '1' '1' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '1' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' 'mitad_N51.jpg'
 'mitad_N59.jpg' 'mitad_Y10.jpg' 'mitad_Y102.jpg' 'mitad_Y106.jpg'
 'mitad_Y108.jpg' 'mitad_Y110.jpg' 'mitad_Y119.jpg' 'mitad_Y121.jpg'
 'mitad_Y123.jpg' 'mitad_Y125.jpg' 'mitad_Y127.jpg' 'mitad_Y129.jpg'
 'mitad_Y130.jpg' 'mitad_Y132.jpg' 'mitad_Y134.jpg' 'mitad_Y136.jpg'
 'mitad_Y138.jpg' 'mitad_Y14.jpg' 'mitad_Y141.jpg' 'mitad_Y143.jpg'
 'mitad_Y145.jpg' 'mitad_Y149.jpg' 'mitad_Y16.jpg' 'mitad_Y2.jpg'
 'mitad_Y21.jpg' 'mitad_Y24.jpg' 'mitad_Y26.jpg' 'mitad_Y28.jpg'
 'mitad_Y3.jpg' 'mitad_Y31.jpg' 'mitad_Y33.jpg' 'mitad_Y35.jpg'
 'mitad_Y38.jpg' 'mitad_Y4.jp